** GOAL **

To reproduce the number from the Austen report.  Numbers will be approximate because we will use averages for tuition paid, admin cost, total credits taken per student, etc.

** DEFINITIONS **

* total student credit hours = (number of students) x (credits for course)


** COST **

* cost = (contact hours)/12 x salary # use ave salary by rank

* admin cost = (department head salary/4 + ? (1/2 of george's salary, maybe admin assist, dean)) / (total number of PHYS courses)

* total costs = cost + admin cost + (department budget / total number of PHYS courses) 

* cost per SCH = (total cost) / total student credit hours


** REVENUE **

* revenue = (tuition x (1-discount rate))/(total credits taken) x (credits for course)* enrollment  + lab_fees * enrollment # use average

* average revenue per SCH = sum(revenue)/sum(enrollment * credit_hours)


** MARGIN **

* margin = revenue - cost

* margin per SCH = (revenue per SCH) - (cost per SCH)

* department margin = sum(revenue) - sum(cost)



** PREPARING THE DATA ** 

* we added two columns to the spreadsheet that Angela sent
    * credit hours (column 15)
    * contact hrs (column 16)
    

** ASSUMPTIONS AND SIMPLIFICATIONS **

* We set all salaries to be equal, including adjunct (\$80,000/yr).
* The only difference between adjunct and tenure-track faculty is that tenure-track faculty have 40% overhead applied in addition to the \$80,000 base salary.
* The differences in cost per SCH between departments is due solely to the fraction of adjuncts teaching within the dept.



In [81]:
import xlrd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import sys

In [82]:
# assumptions

# faculty salaries
salary_assistant = 70000.#70000.
salary_associate = 80000.
salary_full = 95000.#95000.
salary_adjunct = 850.#80000./24#850. #per credit hour
salary_visitor = 65000.#80000.#65000.

assistants = ['Bellis','McColgan','Moustakas','Hassel', # Physics
              'Brookins','Vernooy','Swinton','Goldman','Berke','Springer',#Biology
              'Karr','Deyrup',# Computer Science
              'Fryling','DiTursi','Small',# Computer Science
              'Meierdiercks','Kolozsvary', #Env
              'Javaheri', 'Henry' # Math
              ]
              
associates = ['Finn','Cummings','Vernizzi','Rosenberry', # Physics
              'Memmo-Signor','Mason','Harbison','Byrnes', #Biology
              'Hofstein','Tucker','Moriarty','Barnes',"O'Donnell",'Hughes','Kolonko',#Chemistry
              'Lim','Cotler','Berman','Breimer',# Computer Science
              'Ellard', #ENVA
              'Bannon','Krylov',"O'Neill" #Math
              ]
full = ['Coohill','Medsker','Weatherwax', #Physics
        'Woolbright','Angstadt','Helm','Hayden','Wilson','Worthington','Zanetti','Sterne-Marr','LaRow', #Biology
        'Rhoads', # Chemistry
        'Egan','Vandenberg','Horowitz','Flatland','Matthews', # Computer Science
        'Mangun', #Env
        'Kenney', 'Rogers' #Math
       ]
visitors = ['Caldaro','Russell',
           'Rapp', 'Pier','Chaturvedi', #Biology
            'Wos','Barbera','LaGraff','McNamara','Lee','Vanderover','Perry', #Chemistry
            'Liss','Sherman','Mehta','Goldstein','Yates', #Computer Science
            'Bogan', #Environmental Science
            'Smitas', 'Cade', #Math
            ]
adjuncts = ['Broder','Gigante', #Physics 
           #Biology
          "O'Brien",#Chemistry
            'Coco','Mendez','Rivituso','Schindler','Baciewicz', #Computer Science
            'Pipino', #ENVA
            'Kiehle','Mazzone' #Math
           ]

# make a dictionary of faculty and rank
nfaculty = len(assistants) + len(associates) + len(full) + len(visitors)
faculty_salaries = np.zeros(nfaculty,'f')
faculty_salaries[0:len(assistants)] = salary_assistant*np.ones(len(assistants))
faculty_salaries[len(assistants):len(assistants)+len(associates)] = salary_associate*np.ones(len(associates))
faculty_salaries[len(assistants)+len(associates):len(assistants)+len(associates)+len(full)] = salary_full*np.ones(len(full))
faculty_salaries[len(assistants)+len(associates)+len(full):] = salary_visitor*np.ones(len(visitors))
# self.nsadict=dict((a,b) for a,b in zip(self.s.NSAID,arange(len(self.s.NSAID))))
salary_dict = dict((a,b) for a,b in zip(assistants+associates+full+visitors,faculty_salaries))
# student numbers
discount_rate = .65
tuition = 32000.
tuition_paid = tuition * (1.-discount_rate)
revenue_per_sch = tuition_paid/30.

# physics budget
physics_budget = 35000.

department_budget = {'PHYS':35000.,'BIOL':157000.,'CSIS':57000.,'MATH':10000.,'CHEM':68000.,'ENVA':27000.}
sos_budget = 250000.
saint_center_budget = 1700.

austen_cost_per_sch = {'PHYS':346., 'BIOL':411., 'CHEM':360., 'MATH':286., 'CSIS':320., 'ENVA':353.}
austen_revenue_per_sch = {'PHYS':470., 'BIOL':458., 'CHEM':418., 'MATH':393., 'CSIS':517., 'ENVA':509.}
austen_margin_per_sch = {'PHYS':124., 'BIOL':46., 'CHEM':58., 'MATH':107., 'CSIS':196., 'ENVA':155.}
# tuition
tuition = 32000.

#discount rate

discount_rate = 0.6

tuition_paid = tuition*(1-discount_rate)


In [83]:
def readSchedule(file,match_string): # for reading Angela's excel files
    block=[]
    days=[]
    times=[]
    room=[]
    fac=[]
    credit=[]
    course=[]
    mathcourse=[]
    mathtitle=[]
    mathblock=[]
    mathdays=[]
    mathtimes=[]
    credit_hrs = []
    contact_hrs = []
    enrollment = []
    lab_fee = []
    if file.find('.xls') > -1:# reading an excel file
        xdat=xlrd.open_workbook(file)
        sheet=xdat.sheet_by_index(0)
        ncol=sheet.ncols
        nrow=sheet.nrows
        # find course index
        for i in range(nrow):
            coursename=sheet.cell_value(i,5)

            if coursename.startswith(match_string):# | coursename.startswith('ASTR') : #| (coursename.find('SCDV001') > -1)  | (coursename.find('CSIS200') > -1) | (coursename.find('SCDV230') > -1):
                if sheet.cell_value(i,6).find('-Shen') > -1:
                    #print 'skipping shen physics'
                    continue
                if float(sheet.cell_value(i,3)) < 6.: 
                    #print 'skipping low enrollment course'
                    continue
                enrollment.append(sheet.cell_value(i,3))
                course.append(sheet.cell_value(i,5))
                block.append(sheet.cell_value(i,7))
                days.append(sheet.cell_value(i,8))
                times.append(sheet.cell_value(i,9))
                room.append(sheet.cell_value(i,10))
                fac.append(sheet.cell_value(i,11).split(',')[0])
                credit.append(sheet.cell_value(i,12))
                credit_hrs.append(sheet.cell_value(i,14))
                contact_hrs.append(sheet.cell_value(i,15))
                try:
                    lab_fee.append(float(sheet.cell_value(i,13)))
                except ValueError:
                    #print 'no lab fee ',sheet.cell_value(i,13)
                    lab_fee.append(0.)
        #xdat=xlrd.close_workbook(file)
        return block,room,fac,credit,course,days,times,credit_hrs,contact_hrs,enrollment,lab_fee
    else:
        for line in file:
            if line.startswith('CRN'):
                continue
            t=line.split(',')
            try:
                course.append(t[2])
                block.append(t[4])
                room.append(t[7])
                fac.append(t[8])
                credit.append(t[9])
            except:
                print "Error reading line: ",line
                continue
        return block,room,fac,credit,course,days,times,credit_hrs,contact_hrs 

In [84]:
def readScheduleDorr(file,match_string): # for reading F2013-S2015 file from Brad Dorr
    '''
    
    Brad sent a file with contact and credit hours.
    The file contains data for period covered by Austen Group report: Fall 2013 - Spring 2015.
    I am writing this function to read SD_Credits_r1.xlsx
    
    '''
    block=[]
    fac=[]
    credit=[]
    course=[]
    credit_hrs = []
    contact_hrs = []
    enrollment = []
    lab_fee = []
    term = []
    
    xdat=xlrd.open_workbook(file)
    sheet=xdat.sheet_by_index(0)
    ncol=sheet.ncols
    nrow=sheet.nrows
    # find course index
    for i in range(nrow):
        courseprefix=sheet.cell_value(i,3)

        if courseprefix.startswith(match_string):# | coursename.startswith('ASTR') : #| (coursename.find('SCDV001') > -1)  | (coursename.find('CSIS200') > -1) | (coursename.find('SCDV230') > -1):
            
            # skip Shen Physics
            if sheet.cell_value(i,8).find('-Shen') > -1:
                #print 'skipping shen physics'
                continue
                
            # skip independent studies
            if sheet.cell_value(i,10).find('Independent') > -1:
                #print 'skipping independent studies'
                continue
                
            # skip course with enrollment < 6
            if float(sheet.cell_value(i,9)) < 6.: 
                #print 'skipping low enrollment course'
                continue
            
            enrollment.append(sheet.cell_value(i,9))
            course.append(sheet.cell_value(i,3)+sheet.cell_value(i,4))
            term.append(sheet.cell_value(i,1))
            block.append(sheet.cell_value(i,5))
            fac.append(sheet.cell_value(i,14).split(',')[0])
            credit.append(sheet.cell_value(i,6))
            try:
                t = float(sheet.cell_value(i,6))
                credit_hrs.append(sheet.cell_value(i,6))
            except ValueError: # some are blank - use column Student Credit instead
                print 'Using column 13 for credit_hrs',sheet.cell_value(i,3)+sheet.cell_value(i,4), sheet.cell_value(i,6),sheet.cell_value(i,12)
                
                credit_hrs.append(float(sheet.cell_value(i,12)))
            contact_hrs.append(sheet.cell_value(i,11))
                          
            try:
                lab_fee.append(float(sheet.cell_value(i,8)))
            except ValueError:
                #print 'no lab fee ',sheet.cell_value(i,13)
                lab_fee.append(0.)
        #xdat=xlrd.close_workbook(file)
    return block,fac,credit,course,credit_hrs,contact_hrs,enrollment,lab_fee,term
    

In [85]:
class department:
    def __init__(self,prefix,infile):
        self.prefix = prefix
        if prefix.find('PHYS') > -1: # read file twice to get PHYS and ASTR courses
            block,fac,credit,course,credit_hrs,contact_hrs,enrollment,lab_fee,term = readScheduleDorr(infile,prefix)
            t = readScheduleDorr(infile,'ASTR')
            self.faculty = np.array((fac + t[1]),'S16')
            self.course = np.array((course + t[3]),'S7')
            try:
                self.credit_hrs = np.array((credit_hrs + t[4]),'f')
            except ValueError:
                print 'Value Error when converting credit_hrs to float'
                print credit_hrs
                sys.exit()
            
            self.contact_hrs = np.array((contact_hrs + t[5]),'f')
            self.enrollment = np.array((enrollment + t[6]),'f')
            self.lab_fee = np.array((lab_fee + t[7]),'f')
            self.term = np.array((term + t[8]),'S11')
        else:
            block,fac,credit,course,credit_hrs,contact_hrs,enrollment,lab_fee,term = readScheduleDorr(infile,prefix)
            self.faculty = np.array((fac),'S16')
            self.course = np.array((course),'S7')
            try:
                self.credit_hrs = np.array((credit_hrs),'f')
            except ValueError:
                print 'Value Error when converting credit_hrs to float'
                print credit_hrs
                sys.exit()
            self.contact_hrs = np.array((contact_hrs),'f')
            self.enrollment = np.array((enrollment),'f')
            self.lab_fee = np.array((lab_fee),'f')
            self.term = np.array((term),'S11')
        self.calc_cost()
        self.calc_revenue()
        self.calc_margin()
        self.austen_cost_per_sch = austen_cost_per_sch[self.prefix]
        self.austen_revenue_per_sch = austen_revenue_per_sch[self.prefix]
        self.austen_margin_per_sch = austen_margin_per_sch[self.prefix]
    def initOld(self,prefix,infile):
        self.prefix = prefix
        if prefix.find('PHYS') > -1: # read file twice to get PHYS and ASTR courses
            block,room,fac,credit,course,days,times,credit_hrs,contact_hrs,enrollment,lab_fee = readSchedule(infile,prefix)
            t = readSchedule(infile,'ASTR')
            self.faculty = np.array((fac + t[2]),'S16')
            self.course = np.array((course + t[4]),'S7')
            self.credit_hrs = np.array((credit_hrs + t[7]),'f')
            self.contact_hrs = np.array((contact_hrs + t[8]),'f')
            self.enrollment = np.array((enrollment + t[9]),'f')
            self.lab_fee = np.array((lab_fee + t[10]),'f')
        else:
            block,room,fac,credit,course,days,times,credit_hrs,contact_hrs,enrollment,lab_fee = readSchedule(infile,prefix)
            self.faculty = np.array((fac),'S16')
            self.course = np.array((course),'S7')
            self.credit_hrs = np.array((credit_hrs),'f')
            self.contact_hrs = np.array((contact_hrs),'f')
            self.enrollment = np.array((enrollment),'f')
            self.lab_fee = np.array((lab_fee),'f')
        self.calc_cost()
        self.calc_revenue()
        self.calc_margin()
        self.austen_cost_per_sch = austen_cost_per_sch[self.prefix]
        self.austen_revenue_per_sch = austen_revenue_per_sch[self.prefix]
        self.austen_margin_per_sch = austen_margin_per_sch[self.prefix]
    def calc_cost(self):
        self.cost_per_course = np.zeros(len(self.faculty))
        nhassel = 0
        for i in range(len(self.faculty)):
            #print faculty[i]
            #print salary_dict[self.faculty[i]]
            try:
                if self.faculty[i].find('Hassel') > -1:
                    nhassel += 1
                    if nhassel < 2:
                        self.cost_per_course[i] = salary_dict[self.faculty[i]]/24.*self.contact_hrs[i]*1.4
                    else:
                        self.cost_per_course[i] = self.contact_hrs[i] * salary_adjunct
                else:
                    self.cost_per_course[i] = salary_dict[self.faculty[i]]/24.*self.contact_hrs[i]*1.4
        
            except KeyError: # enter here for adjunct faculty
                #print 'Key error for ',i,self.faculty[i]
                #if faculty[i].find('Gig') > -1:
                #    print 'Key error for ', self.faculty[i]
                #else:
                #    print salary_dict[self.faculty[i]]
                #print 'got here!'
                print self.prefix,': assuming ',self.faculty[i],' is an adjunct professor who teaches ',np.sum(self.contact_hrs[self.faculty == self.faculty[i]]),' contact hrs'
                self.cost_per_course[i] = self.contact_hrs[i]* salary_adjunct# course credit hrs * salary per credit hour
        self.total_cost = np.sum(self.cost_per_course) + department_budget[self.prefix]
        self.cost_per_sch = self.total_cost/np.sum(self.credit_hrs*self.enrollment)
    def calc_revenue(self):
        self.revenue_per_course = tuition_paid/30.*self.credit_hrs*self.enrollment + self.lab_fee*self.enrollment
        self.revenue_per_sch = self.revenue_per_course/(self.enrollment*self.credit_hrs)
        self.total_revenue = np.sum(self.revenue_per_course)
        self.ave_revenue_per_sch = np.sum(self.revenue_per_course)/np.sum(self.enrollment*self.credit_hrs)
    def calc_margin(self):
        self.margin_per_sch = self.revenue_per_sch - self.cost_per_sch
        self.margin_per_course = self.revenue_per_course - self.cost_per_course
        self.total_margin = self.total_revenue - self.total_cost
        self.ave_margin_per_sch = np.sum(self.revenue_per_course - self.cost_per_course)/np.sum(self.enrollment*self.credit_hrs)
    def print_stats(self):
        print '\n%%%%%%%%%%%%%%%%%%%%%%%'
        print self.prefix
        print '%%%%%%%%%%%%%%%%%%%%%%%'
        print 'average cost per class    = %5.2f'%(np.mean(self.cost_per_course))
        print 'average revenue per class = %5.2f'%(np.mean(self.revenue_per_course))
        print 'average margin per class  = %5.2f'%(np.mean(self.margin_per_course))
        print 'average class size        = %3.1f'%(np.mean(self.enrollment))
    def print_stats_alt(self):
        print 'dept cost/course rev/course margin/course enrollment'
        print '%s %5.2f %5.2f   %5.2f %3.1f'%(self.prefix,np.mean(self.cost_per_course), 
                                            np.mean(self.revenue_per_course), np.mean(self.margin_per_course),
                                            np.mean(self.enrollment))
    def print_stats_sch(self):
        print 'dept cost/sch rev/sch margin/sch enrollment'
        print '%s   %5.2f  %5.2f     %5.2f       %3.1f'%(self.prefix,np.mean(self.cost_per_sch),self.ave_revenue_per_sch, self.ave_margin_per_sch,
                                            np.mean(self.enrollment))
    def compare_austen_header(self):
        print '                     Ours    Austen'
    def print_dept(self):
        print '%s'%(self.prefix)
    def compare_austen(self):
        print '   revenue per SCH   %5.1f  %5.1f '%(self.ave_revenue_per_sch,self.austen_revenue_per_sch)
        print '   cost    per SCH   %5.1f  %5.1f '%(self.cost_per_sch,self.austen_cost_per_sch)
        print '   margin  per SCH   %5.1f  %5.1f '%(self.ave_margin_per_sch,self.austen_margin_per_sch)

In [86]:
infile = 'Fall2014.xls'
infile = 'SD_Credits_r1_finn.xlsx' # had to change some blanks in student credit hrs for PHYS 470 & 472
phys = department('PHYS',infile)
bio = department('BIOL',infile)
chem = department('CHEM',infile)
math = department('MATH',infile)
cs = department('CSIS',infile)
enva = department('ENVA',infile)

Using column 13 for credit_hrs PHYS470  1.0
Using column 13 for credit_hrs PHYS472  1.0
Using column 13 for credit_hrs PHYS470  1.0
Using column 13 for credit_hrs PHYS120  4.0
Using column 13 for credit_hrs PHYS472  1.0
Using column 13 for credit_hrs PHYS110  4.0
Using column 13 for credit_hrs PHYS120  4.0
Using column 13 for credit_hrs PHYS120  4.0
Using column 13 for credit_hrs PHYS472  1.0
PHYS : assuming  Broder  is an adjunct professor who teaches  42.0  contact hrs
PHYS : assuming  Broder  is an adjunct professor who teaches  42.0  contact hrs
PHYS : assuming  Broder  is an adjunct professor who teaches  42.0  contact hrs
PHYS : assuming  Broder  is an adjunct professor who teaches  42.0  contact hrs
PHYS : assuming  Broder  is an adjunct professor who teaches  42.0  contact hrs
PHYS : assuming  Gigante  is an adjunct professor who teaches  12.0  contact hrs
PHYS : assuming  Broder  is an adjunct professor who teaches  42.0  contact hrs
PHYS : assuming  Broder  is an adjunct prof

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:94: RuntimeWarning: invalid value encountered in divide


BIOL : assuming  Auker  is an adjunct professor who teaches  25.0  contact hrs
BIOL : assuming  Auker  is an adjunct professor who teaches  25.0  contact hrs
BIOL : assuming  Auker  is an adjunct professor who teaches  25.0  contact hrs
BIOL : assuming  Auker  is an adjunct professor who teaches  25.0  contact hrs
BIOL : assuming  Auker  is an adjunct professor who teaches  25.0  contact hrs
BIOL : assuming  Auker  is an adjunct professor who teaches  25.0  contact hrs
BIOL : assuming  Auker  is an adjunct professor who teaches  25.0  contact hrs
BIOL : assuming  Auker  is an adjunct professor who teaches  25.0  contact hrs
BIOL : assuming  Datta  is an adjunct professor who teaches  6.0  contact hrs
BIOL : assuming  Giarla  is an adjunct professor who teaches  20.0  contact hrs
BIOL : assuming  Giarla  is an adjunct professor who teaches  20.0  contact hrs
BIOL : assuming  Giarla  is an adjunct professor who teaches  20.0  contact hrs
BIOL : assuming  Datta  is an adjunct professor wh

In [87]:
depts = [phys, bio, chem, math, cs, enva]

In [88]:
# what is budget of sos shared between departments
# SoS budget / total credit hours taught in sos
total_credit_hrs = 0
for d in depts:
    total_credit_hrs += np.sum(d.credit_hrs*d.enrollment)
overhead_per_sch = sos_budget/total_credit_hrs
print 'overhead for SOS per SCH = ',overhead_per_sch
print total_credit_hrs

overhead for SOS per SCH =  4.28228845495
58380.0


# Cost

In [89]:
for d in depts:
    print d.print_stats_alt()

dept cost/course rev/course margin/course enrollment
PHYS 12663.19 19571.11   6907.92 19.8
None
dept cost/course rev/course margin/course enrollment
BIOL 14102.71 16488.68   2385.97 18.7
None
dept cost/course rev/course margin/course enrollment
CHEM 11916.11 13578.45   1662.34 18.0
None
dept cost/course rev/course margin/course enrollment
MATH 10034.48 16649.81   6615.33 17.5
None
dept cost/course rev/course margin/course enrollment
CSIS 9130.14 12641.22   3511.08 18.0
None
dept cost/course rev/course margin/course enrollment
ENVA 12659.57 15913.76   3254.18 15.1
None


In [90]:
for d in depts:
    print d.prefix, np.mean(d.contact_hrs),np.median(d.contact_hrs),np.min(d.contact_hrs),np.max(d.contact_hrs), np.mean(d.contact_hrs)*(80000./24)*1.4

PHYS 2.89062 3.0 1.0 4.0 13489.5833333
BIOL 3.07267 3.0 0.0 4.0 14339.1477267
CHEM 2.64901 3.0 1.0 4.0 12362.030824
MATH 2.25862 2.0 1.0 3.0 10540.2301153
CSIS 2.09475 2.0 0.0 4.0 9775.49521128
ENVA 2.81915 3.0 1.0 4.0 13156.0287476


* We set all salaries to be equal, including adjunct (\$80,000/yr).
* The only difference between adjunct and tenure-track faculty is that tenure-track faculty have 40% overhead applied in addition to the \$80,000 base salary.
* The differences in cost per SCH between departments is due solely to the fraction of adjuncts teaching within the dept.
* Based on this, bio is most expensive b/c they have no adjuncts, phys is least expensive b/c we 

# Revenue

Some issues:
* labs have zero credit hours.  Therefore, when you calculate revenue per student credit hours for a lab course, you get infinity.
* to handle this, you could do 

ave_revenue_per_sch = (total_revenue - total_cost)/dot(enrollment,credit_hrs)

**Credit Hours per Course and Revenue per SCH**

In [91]:
for d in depts:
    print d.prefix, np.mean(d.credit_hrs),np.median(d.credit_hrs),np.min(d.credit_hrs),np.max(d.credit_hrs), (d.ave_revenue_per_sch)

PHYS 2.11458 3.0 0.0 4.0 426.667
BIOL 1.55523 0.0 0.0 4.0 426.667
CHEM 1.25166 0.0 0.0 4.0 426.667
MATH 1.77011 1.0 0.0 4.0 426.667
CSIS 1.30594 0.0 0.0 4.0 426.667
ENVA 2.2766 3.0 0.0 4.0 426.667


The mean credit-hours per course is basically telling us the relative fraction of labs offerred by each department. Labs are zero credit hours, so they lower the average credit-hrs per course. The more labs offered, the lower the average revenue per course.  However, lab fees help offset the lower margin of lab courses.

# Margin Analysis

In [92]:
# print statistics per student credit hour
for d in depts:
    d.print_stats_sch()

dept cost/sch rev/sch margin/sch enrollment
PHYS   280.04  426.67     150.60       19.8
dept cost/sch rev/sch margin/sch enrollment
BIOL   376.74  426.67     61.74       18.7
dept cost/sch rev/sch margin/sch enrollment
CHEM   381.51  426.67     52.23       18.0
dept cost/sch rev/sch margin/sch enrollment
MATH   258.12  426.67     169.52       17.5
dept cost/sch rev/sch margin/sch enrollment
CSIS   312.55  426.67     118.51       18.0
dept cost/sch rev/sch margin/sch enrollment
ENVA   347.12  426.67     87.25       15.1


# Compare Our Results and Austen Report Results

In [93]:
# compare revenue, cost and margin per student credit hour
depts[0].compare_austen_header()
for d in depts:
    d.print_dept()
    d.compare_austen()

                     Ours    Austen
PHYS
   revenue per SCH   426.7  470.0 
   cost    per SCH   280.0  346.0 
   margin  per SCH   150.6  124.0 
BIOL
   revenue per SCH   426.7  458.0 
   cost    per SCH   376.7  411.0 
   margin  per SCH    61.7   46.0 
CHEM
   revenue per SCH   426.7  418.0 
   cost    per SCH   381.5  360.0 
   margin  per SCH    52.2   58.0 
MATH
   revenue per SCH   426.7  393.0 
   cost    per SCH   258.1  286.0 
   margin  per SCH   169.5  107.0 
CSIS
   revenue per SCH   426.7  517.0 
   cost    per SCH   312.6  320.0 
   margin  per SCH   118.5  196.0 
ENVA
   revenue per SCH   426.7  509.0 
   cost    per SCH   347.1  353.0 
   margin  per SCH    87.2  155.0 


# Conclusions

Using our simplified model to analyze enrollment data for the 2014 Fall semester, Physics has the largest margin per student credit hour of all departments within the school of science.  This is due to a few factors.  

COST: Physics has a larger fraction of adjuct faculty.  In our simplified model, full-time faculty and adjuncts receive the same salary, so the main financial advantage of adjunct faculty is that they do not have an associated overhead, whereas full-time faculty have a 40 per cent overhead associated with their base salary.

REVENUE:

** outcome of 6/15 meeting - for next time**
* check faculty ranks
* check spreadsheets
* automate credit_hrs vs contact_hrs
* read in multiple semesters 